This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [29]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

#importing libraries
import pandas as pd 
import numpy as np
import datetime as dt

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [30]:
#from io import StringIO

'''n = 20
with open('data_000637.txt', 'r') as f:
    head = ''.join(f.readlines(n))

df = pd.read_csv(StringIO(head))'''

file_name = "data_000637.txt"
data = pd.read_csv(file_name)

print(data)


         HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0           1     0          123  3869200167        2374        26
1           1     0          124  3869200167        2374        27
2           1     0           63  3869200167        2553        28
3           1     0           64  3869200167        2558        19
4           1     0           64  3869200167        2760        25
...       ...   ...          ...         ...         ...       ...
1310715     1     0           62  3869211171         762        14
1310716     1     1            4  3869211171         763        11
1310717     1     0           64  3869211171         764         0
1310718     1     0          139  3869211171         769         0
1310719     1     0           61  3869211171         762        18

[1310720 rows x 6 columns]


In [32]:
#since n max rows = 1310720

n = 1210720
file_name = "data_000637.txt"
data = pd.read_csv(file_name, nrows = n)

print(data)


         HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0           1     0          123  3869200167        2374        26
1           1     0          124  3869200167        2374        27
2           1     0           63  3869200167        2553        28
3           1     0           64  3869200167        2558        19
4           1     0           64  3869200167        2760        25
...       ...   ...          ...         ...         ...       ...
1210715     1     0           51  3869210501        1716        10
1210716     1     0          139  3869210501        1721         0
1210717     1     1           54  3869210501        1713         1
1210718     1     0           56  3869210501        1719        26
1210719     1     0           49  3869210501        1733        26

[1210720 rows x 6 columns]


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [31]:
'''itime = dt.datetime.now()
print("Begin time:", itime)

# the one-liner command
data['TIMENS'] = data['TDC_MEAS'] * 25 / 30 + data['BX_COUNTER'] * 25

ftime = dt.datetime.now()
print("End time:", ftime)
print("Elapsed time:", ftime - itime)

data'''

'''j= ''
BX = []
counter = 0
ORBIT = []
data = pd.read_csv(file_name, nrows = 100)
for i in data['ORBIT_CNT']:
    ORBIT.append(i)

for i in range(len(ORBIT)):
    if i+1 < len(ORBIT):
        while ORBIT[i+1]==ORBIT[i] :
            print(data['BX_COUNTER'])
        counter += 1
    else:
        break
        
    '''BX.append(0)
    print("hello")
    while i+1 == j:
        print("hello")
        BX[counter] += data['BX_COUNTER']
    counter += 1'''

print(ORBIT)'''
    
    #no clue
        

SyntaxError: invalid syntax (2461209891.py, line 29)

3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [33]:
initialT = dt.datetime.now()
print("Begin time:", initialT)

data = pd.read_csv("data_000637.txt")

finalT = dt.datetime.now()
print("End time:", finalT)
print("Elapsed time:", finalT - initialT)

Begin time: 2021-11-19 15:51:06.326797
End time: 2021-11-19 15:51:06.567294
Elapsed time: 0:00:00.240497


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [34]:
data["absolute time [ns]"] = data['TDC_MEAS'] * 25 / 30 + data['BX_COUNTER'] * 25 #idk what should I use to compute :)

data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,absolute time [ns]
0,1,0,123,3869200167,2374,26,59371.666667
1,1,0,124,3869200167,2374,27,59372.500000
2,1,0,63,3869200167,2553,28,63848.333333
3,1,0,64,3869200167,2558,19,63965.833333
4,1,0,64,3869200167,2760,25,69020.833333
...,...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14,19061.666667
1310716,1,1,4,3869211171,763,11,19084.166667
1310717,1,0,64,3869211171,764,0,19100.000000
1310718,1,0,139,3869211171,769,0,19225.000000


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [49]:
#data

data1 = data.groupby('TDC_CHANNEL').sum('TDC_MEAS')
data1.sort_values(by = 'TDC_MEAS', ascending = True)


data1

,HEAD,FPGA,ORBIT_CNT,BX_COUNTER,TDC_MEAS,absolute time [ns]
TDC_CHANNEL,,,,,,
1,29653,28438,114733571761779,52999263,430533,1.325340e+09
2,34271,32669,132601566229845,61017792,499099,1.525861e+09
3,23463,21970,90783184937635,41678301,340104,1.042241e+09
4,28755,26403,111259024921199,51473537,416945,1.287186e+09
5,16435,15003,63590404218625,29099531,238778,7.276873e+08
...,...,...,...,...,...,...
129,37,35,143160633554,67804,612,1.695610e+06
130,71,38,274713643273,124418,993,3.111278e+06
137,68,36,263106045511,123920,748,3.098623e+06


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel